# Estimativa de custo e tempo de transporte áereo por distância

In [1]:
from selenium import webdriver
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
import time
import pandas as pd

In [2]:
id_aeroporto = pd.read_excel('data/excel/xlsx/id_aeroporto.xlsx', engine='openpyxl')

In [9]:
id_aeroporto.head(1)

,SIGLA,ID_AEROPORTO,UF,LATITUDE,LONGITUDE,CLASSIF,PASSAGEIROS_ULTIMO_ANO,ORIGENS,DESTINOS
0,SBCF,SBCF - CONFINS,MG,-19.624444,-43.971944,Classe IV,11765147,"['KMIA - MIAMI, FLORIDA', 'SBAX - ARAXÁ', 'SBB...","['SBGL - RIO DE JANEIRO', 'SBAX - ARAXÁ', 'SBB..."
